In [20]:
# Import necessary modules
import os
import time
import jieba # Get stats of the Chinese Sentences   
import pandas as pd
from gtts import gTTS
from playsound import playsound
from datetime import datetime
from selenium import webdriver
from chinese import ChineseAnalyzer # Get stats of the Chinese Sentences
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# Define start time
startTime = time.time()
# Define time functions
def convert(seconds): 
    seconds = seconds % (24 * 3600) 
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    return "%dh %02dm %02ds" % (hour, minutes, seconds)
def convertTTS(seconds): 
    seconds = seconds % (24 * 3600) 
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    return "%d hours %02d minutes %02d seconds" % (hour, minutes, seconds)
# Define scraping functions
def scrapeDeepL(language):
    # Define the URL to be opened
    url = 'https://www.deepl.com/translator'
    # Define the driver for Selenium to use
    driver = webdriver.Chrome('chromedriver.exe')
    # Open the specfied URL
    driver.get(url)
    # Open the "Select Language" dropdown menu
    buttonSource = driver.find_element_by_xpath('//*[@id="dl_translator"]/div[3]/div[2]/div[1]/div[1]/div').click()
    if language == '中文':
        # Click the Chinese button
        button = driver.find_element_by_xpath('//*[@id="dl_translator"]/div[3]/div[2]/div[1]/div[1]/div/div/button[12]').click()
    else:
        # Click the Japanese button
        button = driver.find_element_by_xpath('//*[@id="dl_translator"]/div[3]/div[2]/div[1]/div[1]/div/div/button[11]').click()
    # Find the text field
    textFieldSource = driver.find_element_by_xpath('//*[@id="dl_translator"]/div[3]/div[2]/div[1]/div[2]/div[1]/textarea')
    # Type the text into the text field
    test = source.loc[0:0].to_string(index = False, columns = None, header = False)
    textFieldSource.send_keys(source['transformed'])
    # Pause to avoid errors
    time.sleep(5)
    # Store the translated text as a string
    translatedText = driver.find_element_by_xpath('//*[@id="target-dummydiv"]')
    translatedText_content = translatedText.get_attribute('innerHTML').strip()
    # Sleep to avoid errors
    time.sleep(2)
    # Clean the output text and print
    textClean = translatedText_content.replace("&nbsp;", "")
    # Sleep to avoid errors
    time.sleep(2)
    # Close the browser window
    driver.quit()
    # Turn the scraped string into a list
    sentences = textClean.split('.')
    sentencesClean = []
    for element in sentences:
        sentencesClean.append(element + ".")
    return sentencesClean
def scrapePapago(language):
    # Define the URL to be opened
    url = 'https://papago.naver.com/'
    # Define the driver for Selenium to use
    driver = webdriver.Chrome('chromedriver.exe')
    # Open the specfied URL
    driver.get(url)
    # Sleep to avoid errors
    time.sleep(2)
    # Find the Text Field and type the sentences
    textField = driver.find_element_by_xpath('//*[@id="txtSource"]')
    test = source.loc[0:0].to_string(index = False, columns = None, header = False)
    for row in source['transformed']:
        textField.send_keys(row)
        textField.send_keys(Keys.RETURN)
    # Sleep to avoid errors
    time.sleep(3)
    # Store translated Text as a string
    translatedText = driver.find_element_by_xpath('//*[@id="txtTarget"]')
    output = translatedText.text
    # Sleep to avoid errors
    time.sleep(1)
    # Close the browser window
    driver.quit()
    # Create a list from the string output
    sentencesClean = output.split("\n")
    return sentencesClean
# Create a DataFrame of the sentences to be translated
source = pd.read_excel('source.xlsx', header = None)
source.columns = ['origin', 'transformed', 'language']
# Get the project name
project = source.language[1]
# Get the language
language = source.language[0]
# Drop the language column
source = source.drop(columns = 'language')
# Drop nulls
source = source.dropna(axis = 0)
# Determine which scraper to use
if language == '한국어':
    sentencesClean = scrapePapago(language)
else:
    sentencesClean = scrapeDeepL(language)
# Create a DataFrame
cols = [language, 'English']
df = pd.DataFrame(columns = cols)
df['English'] = sentencesClean
df[language] = source['transformed']
# Drop nan rows
df.dropna(inplace = True, axis = 0)
# Export DataFrame as an XLSX to Google Drive\Work\Translation folder
df.to_excel('{}\{}.xlsx'.format(language, project), index=False)
os.startfile('{}\{}.xlsx'.format(language, project))
# Alert the user to the script's completion
tts = gTTS(text = "I have finished translating the sentences in {}. Please fix my mistakes, master.".format(convertTTS(time.time() - startTime)), lang = "en")
tts.save("tts.mp3")
playsound("tts.mp3")
os.remove("tts.mp3")